# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
Pkg.activate("..") # activate the parent environment
using BootstrapVirasoro
n_threads = Threads.nthreads()
if n_threads == 1
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`")
end

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro`


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}


In [4]:
setprecision(BigFloat, 30, base=10)
c = CC(β=1/(big"0.8" + big"0.1"*im))
Δmax = 40.

LoopFields = Dict(
    :On => vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=r, s=s) for r in 1//2:1:15 for s in -1+1//(2r):1//r:1 if (r*s)%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:15]
    ),
    :PSUn => vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    ),
    :Potts => vcat(
        [Field(c, r=r, s=s) for r in 2:15 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
)

LoopSpectrum(model, Δmax) = Spectrum(LoopFields[model], Δmax, interchiral=true);

## Bootstrap equations

We numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [6]:
indices = ((1 // 2, 0), (1 // 2, 0), (1, 0), (1, 0))

function solve(indices, Δmax; signature = Dict(:s => 0, :t => 0, :u => 0), diags = Dict())
    fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(fields..., Δmax=Δmax)
    SOn = LoopSpectrum(:On, Δmax)

    Schan_On = ChannelSpectra(
        co, SOn,
        signature=signature
    )

    for (chan, VP) in diags
        add!(Schan_On, VP)
    end

    b = BootstrapSystem(Schan_On)
    compute_linear_system!(b)
    solve!(b, precision_factor=1)
    return b
end

b = solve(indices, Δmax, signature=Dict(:s => 1, :t => 1//2, :u => 3//2))

┌ Info: system size: (158, 152)
└ @ BootstrapVirasoro /Users/paulroux/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/bootstrap_equations.jl:159


BootstrapSystem{Complex{BigFloat}, ChannelSpectrum{Complex{BigFloat}}}(Complex{BigFloat}[0.27357347883158966705963166532456 + 0.4405378653824866819199712608679im, 0.39643374656872654160366664655157 + 0.32538418241633915162225321182632im, 0.14638441128665502510841633920791 + 0.44004496393556102873390045715496im, 0.37070433864418139702223697895533 + 0.10078527106748476349640242233363im, 0.2745967306134557972541188064497 + 0.25949556916146510054943519207882im, 0.18448799973375798466612707215972 + 0.22680939729809565519325076365931im, 0.32951327151456033526244482345646 + 0.20333113415737505413893870809261im, 0.39507756025906115571189047841472 + 0.48828449239691745775715503441461im, 0.37343099361370912792068565977388 + 0.16264986258506231031617517146515im, 0.10050967287693156337891053908606 + 0.41670229112222145761990077517112im  …  0.12643923139340107475447894103127 + 0.1507766055382203929102757911096im, 0.43160986115734212287264881524607 + 0.4612683516057436428070559486514im, 0.4776399933

In [ ]:
b.consts